# 운송 비용확인

## 가설: 운송 비용을 낮출 효율적인 운송경로가 존재한다

In [54]:
import pandas as pd
import numpy as np

In [2]:
# 운송 경로 및 운송 비용 데이터 파일 불러오기
df_tr = pd.read_csv('trans_route.csv', index_col = '공장')
df_tc = pd.read_csv('trans_cost.csv', index_col= '공장')

In [3]:
df_tr

,F1,F2,F3,F4
공장,,,,
W1,15,15,0,5
W2,5,0,30,5
W3,10,15,2,15


In [4]:
df_tc

,F1,F2,F3,F4
공장,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


In [13]:
# 총 비용 함수 만들기

def trans_cost(df_tr, df_tc):

    cost = 0

    for i in range(len(df_tc.index)):
        for j in range(len(df_tr.columns)):
            cost += df_tr.iloc[i][j] * df_tc.iloc[i][j]

    return cost

In [17]:
total_cost = trans_cost(df_tr, df_tc)
print('총 운송 비용:' , str(total_cost) + '만원' )

총 운송 비용: 1493만원


## 제약 조건 설정

In [18]:
df_demand = pd.read_csv('demand.csv')
df_supply = pd.read_csv('supply.csv')

In [19]:
df_demand

,F1,F2,F3,F4
0,28,29,31,25


In [20]:
df_supply

,W1,W2,W3
0,35,41,42


In [21]:
df_tr

,F1,F2,F3,F4
공장,,,,
W1,15,15,0,5
W2,5,0,30,5
W3,10,15,2,15


In [46]:
#수요측 제약 조건
for i in range(len(df_demand.columns)):

    temp_sum = sum(df_tr[df_demand.columns[i]])
    print(str(df_demand.columns[i]) + '으로 운송량:' + str(temp_sum) +  "(수요량:" + str(df_demand.iloc[0][i]) + ")")
    
    if temp_sum >= df_demand.iloc[0][i]:
        print('수요량 만족')
    
    else:
        print('수요량 불충족, 운송경로 재계산 필요')

F1으로 운송량:30(수요량:28)
수요량 만족
F2으로 운송량:30(수요량:29)
수요량 만족
F3으로 운송량:32(수요량:31)
수요량 만족
F4으로 운송량:25(수요량:25)
수요량 만족


In [53]:
#공급측 제약 조건
for i in range(len(df_supply.columns)):
    temp_sum = sum(df_tr.loc[df_supply.columns[i]])
    print(str(df_supply.columns[i]) + '부터의 운송량:' + str(temp_sum) +  "(공급한계:" + str(df_supply.iloc[0][i]) + ")")
    
    if temp_sum <= df_supply.iloc[0][i]:
        print('공급한계 범위내')
    
    else:
        print('공급한계 초과, 운송경로 재계산 필요')

W1부터의 운송량:35(공급한계:35)
공급한계 범위내
W2부터의 운송량:40(공급한계:41)
공급한계 범위내
W3부터의 운송량:42(공급한계:42)
공급한계 범위내


## 수정한 운송경로 제약 조건 확인

In [55]:
#데이터 불러오기
df_tr_new = pd.read_csv('trans_route_new.csv', index_col ='공장')
df_tr_new

,F1,F2,F3,F4
공장,,,,
W1,15,15,0,0
W2,5,0,30,10
W3,10,15,2,15


In [64]:
df_tr

,F1,F2,F3,F4
공장,,,,
W1,15,15,0,5
W2,5,0,30,5
W3,10,15,2,15


In [57]:
#총 운송 비용 재계산
new_total_cost=trans_cost(df_tr_new, df_tc)
print('변경된 총 운송 비용:' , str(new_total_cost) + '만원' )

변경된 총 운송 비용: 1428만원


In [62]:
## 제약 조건 계산함수

# 수요측 제약 조건
def condition_demand(df_tr, df_demand):
    
    flag = np.zeros(len(df_demand.columns))
    
    for i in range(len(df_demand.columns)):
        temp_sum = sum(df_tr[df_demand.columns[i]])
        
        if (temp_sum >= df_demand.iloc[0][i]):
            flag[i] = 1
    
    return flag

# 공급측 제약 조건

def condition_supply(df_tr, df_supply):
    
    flag = np.zeros(len(df_supply.columns))
    
    for i in range(len(df_supply.columns)):
        temp_sum = sum(df_tr.loc[df_supply.columns[i]])
        
        if (temp_sum <= df_supply.iloc[0][i]):
            flag[i] = 1
    
    return flag   

In [63]:
print('수요조건 계산결과:' + str(condition_demand(df_tr_new, df_demand)))
print('공급조건 계산결과:' + str(condition_supply(df_tr_new, df_supply)))

수요조건 계산결과:[1. 1. 1. 1.]
공급조건 계산결과:[1. 0. 1.]


- 새로운 운송경로를 통해 W1에서 F4로 가는 운송을 줄이고, 그만큼 W2에서 F4로위 운송을 늘리는 경로
- 새로운 운송경로의 총 비요은 1428만원으로 원래 운송 비용(1493만원)보다 65만원 절감 되지만, W2의 공급한계를 넘어버림